**A PET Scan Simulator**

By: Nicholas Trigger and James Bradley


Using Ideas from:
    Mahoney, D., Huang, S. C., Ricci, A. R., Mazziotta, J. C., Carson, R. E., Hoffman, E. J., & Phelps, M. E. (1987). A Realistic Computer-Simulated Brain Phantom for Evaluation of PET Charactenstics. IEEE transactions on medical imaging, 6(3), 250–257. https://doi.org/10.1109/TMI.1987.4307834

In [14]:
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from PIL import Image
from Helpers.image_to_phantom import img_to_phantom, get_image_info

ex:

In [15]:
image_info = get_image_info()